# Contact

**Name**            :   Ayush Pandey

**University**      :   [Indian Institute of Technology (IIT), Kharagpur](http://iitkgp.ac.in)

**Email**           :   ayushpandey.iitkgp@gmail.com

**IRC Handle**      :   KrishnaKanhaiya at freenode.net

**Github Username** :   [Ayush-iitkgp](https://github.com/Ayush-iitkgp)

**Mentor**          :   [Christopher Rackauckas](https://github.com/ChrisRackauckas)

# The Project

## The Problem and Motivations
The aim of the project is to add different optimization and bayesian techniques to estimate the parameterd of the differential equation in DiffEqParamEstim.jl package and do a comprehensive study of the pros and cons of the different algorithms to add it to the documentation.

Differential equation models are widely used in many scientific fields that include engineering, physics and biomedical sciences. The so-called “forward problem” that is the problem of solving differential equations for given parameter values in the differential equation models has been extensively studied by mathematicians, physicists, and engineers. 

However, the **inverse problem**, the problem of parameter estimation based on the measurements of output variables, has not been well explored using modern optimization and statistical methods. Parameter estimation aims to find the unknown parameters of the model which give the best fit to a set of experimental data. In this way, parameters which cannot be measured directly will be determined in order to ensure the best fit of the model with the experimental results. This will be done by globally minimizing an objective function which measures the quality of the fit. This inverse problem usually considers a cost function to be optimized (such as maximum likelihood). This problem has applications in systems biology, HIV-AIDS study, and drug dosage estimation.

# The Plan
I propose to implement different algorithms for estimation of the parameters of the differential equations such as:

1. Support for more Kernels for two-stage method
2. Implement support for different distance measures such as (log-)likelihood and expectation maximization
3. Implement regularization techniques such as Tikhonov regularization, LASSO regularization
4. Implement support for meta-heuristic algorithms for global optimum such as enhanced scatter search, genetic algorithm.
5. Implement model based smoothing approach to estimate the parameters
6. Bayesian techniques for parameter estimation using Stan.jl

**Stretch Goals**
1. Implementing support for parameter estimation for stochastic differential equations   
2. Writing native julia bayesian computational engine using Mamba.jl and Klara.jl

I also aim to do a detailed study of the methods and write a proper documentation on where to use the algorithms, precuations to take while using them and the pros and cons of the above methods.

## Mathematical Formulation
First of all, we need to understand the mathematical formulation of the inverse parameter estimation problem before we get into the implementation details.

### General Form of Parameter Estimation Problem

![title](images/General_Parameter_Estimation_Form.png)

where $x \in  \mathbf{R}^{n}$ is the observed state vector, $\theta \in {R}^{m}$ is the vector of unknown parameters, f(.) is a known linear or non-linear function vector. In general, we may not observe x directly but we can observe a function of x, here g(.) is the observation functions that maps the state variable to vector of observable quantities $y \in {R}^{n}$, these are the signals thar can be measured in the experiments.


### Maximum likelihood and cost function
Maximum likelihood and cost function Assuming that the transformed measurements y are contaminated by additive normally distributed uncorrelated random measurement errors i.e. $y_{ijk}$ = $y_{ijk}(x(t_{i}), θ) + e_{ijk}$ where $e_{ijk} $∼$ N(0, σ2_{ijk})$ is the random error with standard deviation $σ_{ijk}$ and $y_{ijk}$ is the measured value, the estimation of the model parameters is formulated as the maximization of the likelihood of the data:

![title](images/Maximum_Likelihood.png)

where $N_{e}$ isthe number of experiments, $N_{y,k}$ is the number of observed compounds in the k-th experiment, and $N_{t,k,j}$ is the number of measurement time points of the j-th observed quantity in the k-th experiment.

From the thoery of maximum likelihood, **the objective is to maximize the likelihood function** and estimate the parameters for which the likelihood function is maximized.

Since the likelihood function consist of product terms, we take the log of the likelihood function and negate it to ease our calculation, thus we get log-likelihood function as follows:

![title](images/Maximum_log_Likelihood.png)

**Note** - The maximization of the likelihood function (4) is equivalent to the minimization of the weighted least squares cost function (5).

where the residual vector R(·) : $R^{Nθ}$ → $R^{ND}$ is constructed from the squared terms by arranging them to a vector. With this, the model calibration problem can be stated as the well-known nonlinear least-squares (NLS) optimization problem:

![title](images/NLS.png)

A θ vector that solves this optimization problem is called the maximum-likelihood estimates of model parameters.

Therefore, the objective of the "**inverse problem**" is to pose the problem as the optimization problem and get the optimum value to estimate the parameters.


# Execution

## 1. Support for more Kernels for two-stage method

This method is inspired from the research paper [Parameter estimation for Differential Equation Models using a Framework of Measurement Error in Regression Models](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2631937/) where the idea is to estimate the parameters of the Ordinary Differential Equations (ODE) using local smoothing approach and a pseudo-least squares.

**Note** - This method only works for ordinary differential equations models.

Let's rewrite the differential equation written above in another form:

![image](images/2_stage_form_ODE.png)

where X(t) = ${ X1(t), …, Xk(t)}^{T}$ is an unobserved state vector, β = $(β1, …, βm)^{T}$ is a vector of unknown parameters, and F(·) = ${F1(·), …, Fk(·)}^{T}$ is a known linear or nonlinear function vector. In practice, we may not observe X(t) directly, but we can observe its surrogate Y(t). For simplicity, assume an additive measurement error model to relate X(t) to the surrogate Y(t), i.e.,

![image](images/2_stage_form_Y.png)

where the measurement error e(t) is independent of X(t) with a covariance matrix $Σ_{e}$.

Suppose X̂′(t) is an estimator of X′(t). Substituting the estimates X̂′(ti), i = 1, …, n, in the ODE equation above, we obtain a regression model:

![image](images/2_stage_form_reg.png)

where Δ($t_{i}$) denotes the substitution error vector, that is Δ($t_{i}$) = X̂′($t_{i}$)− X′($t_{i}$). If X′($t_{i}$) is an unbiased estimator of X′($t_{i}$), Δ($t_{i}$) are errors with mean zero but are not independent. However, if the estimator X′($t_{i}$) is a biased estimator.

The idea is to estimate X(ti) and X′(ti) using regression techniques such as local polynomial regression, smoothing spline and the regression spline.

I have already [implemented](https://github.com/JuliaDiffEq/DiffEqParamEstim.jl/pull/6), two-stage method which uses local linear regression to estimate X(t) and local quadratic regression to estimate X′(t). 

As a consequence, the estimators X̂(t) and X̂′(t) can be expressed as 

![images](images/Parameter Estimation.png)

where 
![image](images/2_stage_form_reg_W.png)
and K(.) is a symmetric kernel function $K_{h}$(.) = K(./h)/h and h is a proper bandwidth.

Currently, the implementation supports **Epanechnikov**, **Uniform** and **Triangular** kernel functions. I propose to include more kernels such as Quartic, Triweight, Tricube, Gaussian, Cosine, Logistic, Sigmoid function, and Silverman.

I also plan to include the advantages and the disadvantages of this method such as :

**Advantages**
1. Computational efficiency
2. Easing of the convergence problem
3. The initial values of the state variables of the differential equations not required
4. Providing good initial estimates of the unknown parameters for other computationally-intensive methods to further refine the estimates rapidly

**Disadvantage**
1. This method does not converge to the global/local minima as the Non-Linear regression does in the objective function is convex/concave.




## 2. Implement support for different distance measures such as (log-)likelihood
In the present implementation the "build_loss_objective" function only covers the Least-Square objective function and the function defined in [LossFunctions.jl](https://github.com/JuliaML/LossFunctions.jl). It also assumes that the errors in measurement are not correlated i.e

![](images/Maximum_Likelihood_no_covariance.png)
Then the maximum likelihood estimate of x given observations y is the solution to the non-linear least squares problem:
![](images/Maximum_Likelihood_no_covariance_optimal.png)

This type of objective function can easily be constructed using LossFunction.jl. However, there are cases when the error is correlated such as:

![](images/Maximum_Likelihood_covariance_.png)

then the maximum likelihood problem to be solved is

![](images/Maximum_Likelihood_covariance_optimal.png)

Thus, I plan to include the functionality to provide the user with te option to state whether the errors are correlated or not and accordingly build the cost/objective function.

## 3. Regularization techniques such as Tikhonov regularization, LASSO regularization

## 4. Provide support for meta-heuristic algorithms for global optimum such as enhanced scatter search, genetic algorithm

## 5. Implement model based smoothing approach to estimate the parameters


## 6. Bayesian techniques for parameter estimation using Stan.jl

<!-- **Note- ** The **"evaluate"** method in the existing atoms can handle the complex variables. But some atoms are only defined over real domain (for example- max(x,0)) thus, I would need to append the warning messages to such atoms in case the users by mistake call the evaluate method with complex argument. --->

# About Me

## Personal Background

I am a final year graduate student pursuing an Integrated Master of Science(MS) degree in Mathematics and Computing Sciences with specialization in Optimization at IIT Kharagpur, India. I am proficient in C, C++, Python, and Julia.

## Previous Relevant Experience

I am a former Google Summer of the Code student with the Julia Language where I worked on the project titled **Support for complex numbers within Convex.jl**. As a result of our work, we became the first open-source DCP package to support optimization with complex-variables. I have also worked on different projects related to optimization and machine learning. Please find all my projects [here](https://drive.google.com/drive/folders/0B2oOdWdSJWa1RWRsQWdCZ25LUXc). I have taken courses on differential equations such as Partial Differential Equations, Numerical solution of ODE and PDE and Advanced NUmnerical Techniques(Theory and Lab) where I have written MATLAB/C [code](https://github.com/Ayush-iitkgp/Numerical-Solution-of-ODE-and-PDE) for different numerical techniques used in solving ODEs and PDEs. 

## Relevant Courses
* Partial Differential Equations
* Numerical solution of ODE and PDE
* Advanced Numerical Techniques
* Stochastic Processes
* Non-Linear Programming
* Convex Optimization
* Probability & Statistics, Regression, Generalized Linear Models
* Linear Algebra, Programming and Data Structures, Object Oriented System Design


## Answers of listed questions

`1. What do you want to have completed by the end of the program?`

By the end of the program, I want to support different optimization and bayesian techniques used to estimate the parameters of the differential equations given the experimental data. Presently, DiffEqParamEstim only supports Non-Linear Regression technique to estimate the parameters. I aim to implement numerous techniques such as two-stage method, model relaxation technique, regularization, log-likelihood estimation, meta-heuristic techniques for global optimum and bayesian techniques using Stan.jl.

`2. Who’s interested in the work, and how will it benefit them?`

The problem of parameter estimation has extensive use in the following fields:
1. HIV-AIDS viral dynamics
2. Systems biology
3. Drug dosage estimation

`3. What are the potential hurdles you might encounter, and how can you resolve them?`

In order to link Stan with DiffEqParamEstim, I will have to understand how Stan works. I will also need to go through the theory of Markov Chain Monte Carlo and Hamiltonian Monte Carlo to write in-situ Bayesian estimator for parameters of the diffeenetial equations. Lastly, I would need to understand the theoty behind the stochastic differential equations to write an implementation to find it's parameters. 

`4. How will you prioritise different aspects of the project like features, API usability, documentation and robustness?`

Please refer to the "Timeline" section where I have described in details about my plans to tackle different aspects of the project.

`5. Does your project have any milestones that you can target throughout the period?`

Yes, before JuliaCon 2017, I would have implemented the various optimization and regualarization algorithms to estimate the parameters and would be ready for testing for the Julia community.

`6. Are there any stretch goals you can make if the main project goes smoothly?`

Yes, I would try to implement native julia computational engine for Bayesian estimation using tools like Mamba.jl or Klara.jl and the inverse problem for Stochastic Differential Equations based on the paper "[Control of Stochastic and Induced Switching in Biophysical Networks](http://journals.aps.org/prx/abstract/10.1103/PhysRevX.5.031036)"

`7. What other time commitments, such as summer courses, other jobs, planned vacations, etc., will you have over the summer?`

I expect to work full time on the project that is 30 or more hours a week.


## Contribution to Open-Source Projects
* Please find the list of all Pull Requests to Convex.jl repository [here](https://github.com/JuliaOpt/Convex.jl/pulls?q=is%3Apr+author%3AAyush-iitkgp+is%3Aclosed).


* Added two-stage method [#6](https://github.com/JuliaDiffEq/DiffEqParamEstim.jl/pull/6)


* [Improved DiffEqDocs](https://github.com/JuliaDiffEq/DiffEqDocs.jl/pulls?q=is%3Apr+author%3AAyush-iitkgp+is%3Aclosed).

## Experience with Julia
I have been using Julia for last one year. In terms of functionality, I like Julia because of its **multiple dispatch** feature as it lets me overload operators with a lot of ease than other programming languages.

But the most astonishing feature of Julia is that it is empowering. In other high-level languages, the users can not be developers because developing new packages in those language require the users to know the intricacies of low-level language whereas in Julia, users can develop packages for their needs in Julia itself without compromising with the speed. 

# Timeline (tentative)

### Community Bonding period (22nd April - 22nd May) 

My summer vacation will start from 30th of April. During this period, I would want to get myself more familiarized with the source code of Convex.jl. I have in particular 2 issue in mind which I would like to send the pull request to namely:

* `Issue multiplying expressions with matrices.`[#122](https://github.com/JuliaOpt/Convex.jl/issues/122)

I believe solving the above issue would help me strengthen my understanding of source code and make myself comfortable with contributing to the package.

* `Missing kron for convex programming variables.`[#57](https://github.com/JuliaOpt/Convex.jl/issues/57)

This would be the first step towards supporting complex numbers in Convex.jl, **kron** atom is used in Chebychev design of an FIR filter given a desired frequency response. Thus implementing this feature would make Convex.jl useful in the applications related to filter design

### Week 1
**Goal:** *Implement support for complex variable and complex SDP in Convex.jl*

I plan to implement the 6 step procedure described in the "Execution" section on weekly basis. We would only be able to merge the code into the existing package after all the 6 steps have been implemented. I would start by making changes in variable.jl file. The end outcome of this week would be that the users would be able to declare a variable as complex-domain variable.

### Week 2
**Goal:** *Overload dot operator and extend minimize/maximize API*

In this week, I would overload the multiply dot operator to support complex multiplication and extend the present API for expressing real SDPs to complex SDPs such that it becomes easy for users to express complex SDPs in Convex.jl.

### Week 3 and 4
**Goal:** *Implement new functions for transformation $\tau$*

These 2 weeks would be required to implement step 3 by making changes in problem.jl file. I would also write new routines to convert complex SDPs to real SPDs.

### Week 5
**Goal:** *Integration of existing solvers via MathProgBase interface*

During this week, I would work to integrate the existing solvers like SCS, Mosek via MathProgBase interface to solve the transformed real SDP, I would also need to change the default setting in Convex.jl so that the output from the solver is not outputted to the user.

### Week 6 and 7
**Goal:** *Convert the solution of the real SDP into corresponding complex SDP*

During these 2 weeks, I implement a new function which would take the primal values and the optimum values of the real SDP and convert to the corresponding complex SDP using the transformation $\tau^{-1}$.

### Week 8
**Goal:** *Display the complex domain solution to the user*

During this week, I would be writing codes to use the existing machinery in Convex.jl to output the optimum value of the objective function and the value of the primal variables to the users once the optimization is complete.

At the end of this week, the support for the complex-domain problems in Convex.jl would be ready for testing for the Julia community.

### Week 9 and 10
**Goal:** *Documentation, Notebooks, Bug fixes*

By this time I will make sure that none of the above implementations has introduced any bug and are complete by documentation as well as testing. I will extend this period to my Future Work as writing example notebooks and
preparing for a major release of the package.

### Week 11 and 12
**Goal:** *Incorporate changes from the feedback given by Julia community and work on presolve routine*

During these two weeks, I plan to work on the suggestions given by the Julia community about the new feature I would be adding. I would try to have extensive discussion with the community during this period and incorporate changes so as to make Convex.jl more robust and user-friendly and ready to be released. If time permits, I also plan to understand the existing SDP solvers like SCS and Mosek in detail and try to figure what could be better ways than the existing ones so that we could make our presolve routine better. I would also be required to read and understand the existing presolve routines for Linear Programming Problems so that I could think on similar lines.

### End-Term evaluation
**Goal:** *Working towards the publication*

Buffer period for any lagging work. I also aim to work towards writing a research paper during this period under the guidance of my mentors.


# References
[1]. [Parameter Estimation for Differential Equation Models Using a Framework of Measurement Error in Regression Models](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2631937/)

[2]. Invariant semidefinite programs. http://arxiv.org/pdf/1007.2905v2.pdf

[3]. Convex Optimization in Julia. http://arxiv.org/pdf/1410.4821.pdf

[4]. [#103](https://github.com/JuliaOpt/Convex.jl/issues/103) Support for complex variables.

[5]. [#191](https://github.com/cvxgrp/cvxpy/issues/191) Add complex variables.

[6]. [Chebychev design of an FIR filter given a desired frequency response](http://nbviewer.jupyter.org/github/cvxgrp/cvxpy/blob/master/examples/notebooks/WWW/fir_chebychev_design.ipynb)

<!--- ipython nbconvert --to FORMAT notebook.ipynb --->